In [1]:
import dv_processing as dv
import sys
import cv2
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
from src.data.events_encoding import accumulate_dv_encode
from src.data.preprocessing import *
from src.models.convlstm import ConvLSTM


In [2]:
raw_path = "../data/raw_data/raw5.aedat4"
encoding_func = accumulate_dv_encode
duration = 10
sequence_length = 300
steps=100
frames = []
capture = dv.io.MonoCameraRecording(raw_path)
slicer = dv.EventStreamSlicer()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def slicing_callback(events: dv.EventStore):
        frame = encoding_func(events, capture.getEventResolution())
        frames.append(frame)

slicer.doEveryTimeInterval(timedelta(milliseconds=duration), slicing_callback)

while capture.isRunning():
    events = capture.getNextEventBatch()
    if events is not None:
        slicer.accept(events)
preprocessed_frames = preprocess_frames(frames, target_size=(32,32))

sequences = []
start_indices = []

for i in range(0, len(frames) - sequence_length + 1, steps):
    sequences.append(torch.stack(preprocessed_frames[i:i+sequence_length]))
    start_indices.append(i)

checkpoint_path = '../saved_models/convlstm_accumulator/epoch=61-step=434.ckpt'
model = ConvLSTM.load_from_checkpoint(checkpoint_path, in_channels=1, feature_size=64).to(device)


In [3]:
predictions = []

# Use voting mechanism to get the final prediction
for start_index, seq in zip(start_indices, sequences):
    seq = seq.unsqueeze(0).to(device)    
    output = model(seq) # (1, 300, 2) (batch, sequence_length, 2)
    pred = output.argmax(dim=-1).squeeze() #  (300)
    pred = pred.cpu().numpy()
    predictions.append((start_index, pred))

# Aggregate predictions using voting
final_pred = np.zeros(len(preprocessed_frames), dtype=int)
vote_counts = np.zeros(len(preprocessed_frames), dtype=int)

for start_index, pred in predictions:
    for i in range(len(pred)):
        final_pred[start_index + i] += pred[i]
        vote_counts[start_index + i] += 1

# Average the votes
for i in range(len(final_pred)):
    if vote_counts[i] > 0:
        final_pred[i] = round(final_pred[i] / vote_counts[i])

# Print final predictions
print(final_pred)
         


[0 0 0 ... 0 0 0]


In [ ]:
import matplotlib.animation as animation

def visualize_sequence(sequence, labels):
    """
    Visualize the video sequence with corresponding labels.
    
    Args:
        sequence: List of frames of shape (height, width)
        labels (torch.Tensor): Labels for each frame in the sequence of shape (sequence_length)
    """
    fig, ax = plt.subplots()
    ax.set_title('Predictions for the video sequence')
    frame_text = ax.text(0.5, 1.05, '', transform=ax.transAxes, ha='center')

    def update(frame_idx):
        ax.imshow(sequence[frame_idx], cmap='gray', vmin=0, vmax=255)
        # Clear the previous label
        for text in ax.texts:
            text.set_visible(False)
        ax.text(0.5, -0.1, f'Predictions: {labels[frame_idx]}', transform=ax.transAxes, ha='center')

    ani = animation.FuncAnimation(fig, update, frames=len(sequence), interval=1000/30) 
    ani.save('sequence2.mp4', writer='ffmpeg', fps=30)
    plt.show()

visualize_sequence(frames, final_pred)


KeyboardInterrupt: 

In [18]:
import time
import cv2

# Open any camera
reader = dv.io.MonoCameraRecording("../data/raw_data/raw5.aedat4")

# Initialize the previous event timestamp
previous_timestamp = None

# Run the loop while camera is still connected
while reader.isRunning():
    events = reader.getNextEventBatch()
    if events is not None:
        current_timestamp = events[0].timestamp()

        if previous_timestamp is not None:
            time_diff = (current_timestamp - previous_timestamp) / 1e6  # Convert microseconds to seconds

            time.sleep(time_diff)
        print({events.size()})
        previous_timestamp = current_timestamp
        frames = accumulate_dv_encode(events, reader.getEventResolution())

        cv2.imshow("Frames", frames)
    


{2}
{2}
{1}
{1}
{3}
{3}
{6}
{1}
{3}
{1}
{1}
{1}
{3}
{1}
{2}
{4}
{1}
{1}
{1}
{4}
{1}
{1}
{1}
{3}
{1}
{1}
{1}
{2}
{1}
{1}
{1}
{1}
{1}
{1}
{2}
{1}
{1}
{3}
{1}
{1}
{1}
{1}
{1}
{1}
{1}
{2}
{1}
{1}
{1}
{2}
{2}
{1}
{5}
{8}
{3}
{2}
{3}
{6}
{1}
{1}
{1}
{2}
{2}
{1}
{1}
{2}
{1}
{1}
{2}
{6}
{1}
{1}
{2}
{1}
{1}
{1}
{1}
{2}
{1}
{3}
{1}
{2}
{2}
{1}
{4}
{3}
{2}
{1}
{2}
{2}
{2}
{1}
{1}
{2}
{2}
{3}
{1}
{1}
{2}
{1}
{3}
{2}
{1}
{1}
{1}
{4}
{1}
{3}
{2}
{1}
{1}
{1}
{2}
{1}
{1}
{1}
{1}
{2}
{1}
{4}
{5}
{6}


KeyboardInterrupt: 

In [17]:
from datetime import timedelta

store1 = dv.data.generate.uniformEventsWithinTimeRange(10000, timedelta(milliseconds=10), (100, 100), 10)
store1.size()

10